<a href="https://colab.research.google.com/github/Iditc/NLP/blob/main/Markov_Model_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [3]:
input_files = [
    'edgar_allan_poe.txt',
    'robert_frost.txt'
]

In [4]:
!head edgar_allan_poe.txt

LO! Death hath rear'd himself a throne
In a strange city, all alone,
Far down within the dim west
Where the good, and the bad, and the worst, and the best,
Have gone to their eternal rest.
 
There shrines, and palaces, and towers
Are not like any thing of ours
Oh no! O no! ours never loom
To heaven with that ungodly gloom!


In [5]:
!head robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [6]:
# collect data into lists
input_texts = []
labels = []

for label, f in enumerate(input_files):
  print(f"{f} corresponds to label {label}")

  for line in open(f):
    line = line.rstrip().lower()
    if line:
      line = line.translate(str.maketrans('', '', string.punctuation))

      input_texts.append(line)
      labels.append(label)

edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


In [7]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [8]:
len(Ytrain), len(Ytest)

(1615, 539)

In [10]:
train_text[:5]

['you must be learned thats what you see in it',
 'studying genealogy with me',
 'that high tone of the spirit which hath striven',
 'with a strange sound as of a harpstring broken',
 'of its own fervour  what had oer it power']

In [11]:
Ytrain[:5]

[1, 1, 0, 0, 0]

In [12]:
idx = 1
word2idx = {'<unk>' : 0}

In [13]:
# populate word2idx
for text in train_text:
  tokens = text.split()
  for token in tokens:
    if token not in word2idx:
      word2idx[token] = idx
      idx += 1

In [14]:
word2idx

{'<unk>': 0,
 'you': 1,
 'must': 2,
 'be': 3,
 'learned': 4,
 'thats': 5,
 'what': 6,
 'see': 7,
 'in': 8,
 'it': 9,
 'studying': 10,
 'genealogy': 11,
 'with': 12,
 'me': 13,
 'that': 14,
 'high': 15,
 'tone': 16,
 'of': 17,
 'the': 18,
 'spirit': 19,
 'which': 20,
 'hath': 21,
 'striven': 22,
 'a': 23,
 'strange': 24,
 'sound': 25,
 'as': 26,
 'harpstring': 27,
 'broken': 28,
 'its': 29,
 'own': 30,
 'fervour': 31,
 'had': 32,
 'oer': 33,
 'power': 34,
 'and': 35,
 'left': 36,
 'lying': 37,
 'where': 38,
 'fell': 39,
 'rejected': 40,
 'well': 41,
 'i': 42,
 'know': 43,
 'now': 44,
 'this': 45,
 'dim': 46,
 'lake': 47,
 'auber': 48,
 'spoke': 49,
 'johns': 50,
 'not': 51,
 'being': 52,
 'safe': 53,
 'to': 54,
 'stay': 55,
 'we': 56,
 'wouldnt': 57,
 'hurt': 58,
 'hen': 59,
 'ought': 60,
 'us': 61,
 'his': 62,
 'doubts': 63,
 'every': 64,
 'open': 65,
 'place': 66,
 'whose': 67,
 'fervid': 68,
 'flickering': 69,
 'torch': 70,
 'life': 71,
 'was': 72,
 'lit': 73,
 'minute': 74,
 'they':

In [15]:
len(word2idx)

2525

In [16]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  test_text_int.append(line_as_int)

In [17]:
train_text_int[100:105]

[[184, 20, 394, 395, 395, 395],
 [35, 396, 212, 18, 397],
 [238, 398, 18, 399],
 [148, 400, 12, 23, 401, 402, 17, 403],
 [8, 404, 174, 42, 405, 127, 12, 381, 18, 406]]

In [18]:
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones((V))

A1 = np.ones((V, V))
pi1 = np.ones((V))

In [19]:
# compute counts for A and Pi
def compute_counts(text_as_int, A, pi):
  last_idx = None
  for idx in tokens:
    if last_idx is None:
      # it's the first word in a sentence
      pi[idx] += 1
    else:
      # the last word exists, so count a transition
      A[last_idx, idx] += 1
    # update last idx
    last_idx = idx

  compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
  compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [21]:
# normalize A and pi so they are valid probability matrices
# convince yourself that this equivalent to the formulas shown before
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()


In [22]:
# log A and pi sinc we dont need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [23]:
# comput priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.3287925696594427, 0.6712074303405573)

In [24]:
from numpy.core.function_base import logspace
# build a classifer
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # its the first token
        logprob +=logpi[idx]
      else:
        logprob +=logA[last_idx, idx]

      # updata last_idx
      last_idx = idx

    return logprob

  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [25]:
# each array must be in order since classes are assumed to index thes ilsts
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [26]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.6712074303405573


In [27]:
Ptest = clf.predict(test_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.6712074303405573
